In [1]:
!apt-get update
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Hit:1 http://ppa.launchpad.net/alessandro-strada/ppa/ubuntu artful InRelease
Hit:2 http://archive.ubuntu.com/ubuntu artful InRelease
Get:3 http://archive.ubuntu.com/ubuntu artful-updates InRelease [88.7 kB]
Get:4 http://security.ubuntu.com/ubuntu artful-security InRelease [83.2 kB]
Get:5 http://archive.ubuntu.com/ubuntu artful-backports InRelease [74.6 kB]
Fetched 247 kB in 0s (263 kB/s)
Reading package lists... Done
gpg: keybox '/tmp/tmpht_xd3hs/pubring.gpg' created
gpg: /tmp/tmpht_xd3hs/trustdb.gpg: trustdb created
gpg: key AD5F235DF639B041: public key "Launchpad PPA for Alessandro Strada" imported
gpg: Total number processed: 1
gpg:               imported: 1
··········


In [2]:
!mkdir -p drive
!google-drive-ocamlfuse drive


fuse: mountpoint is not empty
fuse: if you are sure this is safe, use the 'nonempty' mount option


In [3]:
!ls

CIFAR10CNNTEST	data  datalab  drive  MNIST-data


In [0]:
import sys
pro_path = "drive/CoLab/week6/"
sys.path.append(pro_path)
###replace Colab and Intel-TF101-Class6 according to your folders arrangements

In [0]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import itertools as it
import time

from helpers_05 import (batches, flatten, fully_connected_layer,fully_connected_sigmoid_layer,test_and_show_images)

In [0]:
class MNIST_Model(object):
	# WARNING: 
	# s instead of self is non-standard, for abbreviation purposes
	def __init__(s):  
		graph = tf.Graph()
		with graph.as_default():
			with tf.name_scope('inputs'):
				#s.x_placeholder = tf.placeholder(tf.float32, [None,784], name='x')
				s.x_placeholder = tf.placeholder(tf.float32, [None,3072], name='x')
				s.y_placeholder = tf.placeholder(tf.int32, [None], name='y')

				s.x_keep_prob = tf.placeholder(tf.float32, [], 'x_keep_prob')
				s.dropout_rate = tf.placeholder(tf.float32, [], 'h_keep_prob')
				s.dropout_boolean = tf.placeholder(tf.bool, [], 'dropout_boolean')
				s.momentum      = tf.placeholder(tf.float32, [], 'momentum')
				s.learning_rate = tf.placeholder(tf.float32, [], name='learning_rate')

			with tf.name_scope('preprocess'):
				s.x_dropped = tf.nn.dropout(s.x_placeholder, s.x_keep_prob)
				s.one_hot_labels = tf.one_hot(s.y_placeholder, 10)

			with tf.name_scope('model'):
				s.input_layer = tf.reshape(s.x_placeholder, [-1, 32, 32, 3])
				#s.input_layer = tf.reshape(s.x_placeholder, [-1 ,28 ,28 ,1])

				conv1 = tf.layers.conv2d(inputs=s.input_layer,filters=32, kernel_size=[5, 5],padding="same",activation=tf.nn.relu)
				pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)

				conv2 = tf.layers.conv2d(inputs=pool1,filters=64,kernel_size=[5, 5],padding="same",activation=tf.nn.relu)
				pool2 = tf.layers.max_pooling2d(inputs=conv2, pool_size=[2, 2], strides=2)

				pool2_flat=tf.layers.flatten(inputs=pool2)

				fc_1 = tf.layers.dense(inputs=pool2_flat, units=400, activation=tf.nn.relu)
				dp_1 = tf.layers.dropout(inputs=fc_1, rate=s.dropout_rate, training= s.dropout_boolean)
				fc_2   = tf.layers.dense(inputs=dp_1, units=200, activation=tf.nn.relu)
				dp_2 = tf.layers.dropout(inputs=fc_2, rate=s.dropout_rate, training= s.dropout_boolean)
				fc_3   = tf.layers.dense(inputs=dp_2, units=200, activation=tf.nn.relu)
				dp_3 = tf.layers.dropout(inputs=fc_3, rate=s.dropout_rate, training= s.dropout_boolean)

				s.output = tf.layers.dense(inputs=dp_3, units=10)

			with tf.name_scope('loss'):
				smce = tf.nn.softmax_cross_entropy_with_logits_v2
				s.loss = tf.reduce_mean(smce(logits=s.output, labels=s.one_hot_labels))
				#s.loss = tf.reduce_mean(tf.square(s.output - s.one_hot_labels), name='MSE')

			with tf.name_scope('global_step'):
				global_step = tf.Variable(0, trainable=False, dtype=tf.int64, name="global_step")
				s.inc_step = tf.assign(global_step,global_step + 1)
				#s.global_step = tf.train.get_global_step(graph=graph)

			with tf.name_scope('summaries'):
				"""s.w1_h = tf.summary.histogram('w1_h', s.w1)
				s.b1_h = tf.summary.histogram('b1_h', s.b1)
				s.w1_s = tf.summary.scalar('w1_s', s.w1)
				s.b1_s = tf.summary.scalar('b1_s', s.b1)

				s.w2_h = tf.summary.histogram('w2_h', s.w2)
				s.b2_h = tf.summary.histogram('b2_h', s.b2)
				s.w2_s = tf.summary.scalar('w2_s', s.w2)
				s.b2_s = tf.summary.scalar('b2_s', s.b2)

				s.w3_h = tf.summary.histogram('w3_h', s.w3)
				s.b3_h = tf.summary.histogram('b3_h', s.b3)
				s.w3_s = tf.summary.scalar('w1_s', s.w3)
				s.b3_s = tf.summary.scalar('b1_s', s.b3)

				s.w4_h = tf.summary.histogram('w4_h', s.w4)
				s.b4_h = tf.summary.histogram('b4_h', s.b4)
				s.w4_s = tf.summary.scalar('w1_s', s.w4)
				s.b4_s = tf.summary.scalar('b1_s', s.b4)

				s.w5_h = tf.summary.histogram('w5_h', s.w5)
				s.b5_h = tf.summary.histogram('b5_h', s.b5)
				s.w5_s = tf.summary.scalar('w5_s', s.w5)
				s.b5_s = tf.summary.scalar('b5_s', s.b5)"""

				s.loss_summary = tf.summary.scalar('loss_summary',s.loss)
				s.merged = tf.summary.merge_all()

			with tf.name_scope('train'):
				#s.train = tf.train.GradientDescentOptimizer(s.learning_rate).minimize(s.loss)
				#s.train = tf.train.AdadeltaOptimizer(s.learning_rate,rho=0.95,epsilon=1e-08).minimize(s.loss)
				#s.train = tf.train.AdagradOptimizer(s.learning_rate,initial_accumulator_value=0.1).minimize(s.loss)
				#s.train = tf.train.AdagradDAOptimizer(s.learning_rate,s.inc_step,initial_gradient_squared_accumulator_value=0.1,l1_regularization_strength=0.0,l2_regularization_strength=0.0,).minimize(s.loss)
				#s.train = tf.train.AdamOptimizer(s.learning_rate,beta1=0.9,beta2=0.999,epsilon=1e-08,).minimize(s.loss)
				#s.train = tf.train.FtrlOptimizer(s.learning_rate).minimize(s.loss)
				#s.train = tf.train.ProximalGradientDescentOptimizer(s.learning_rate,l1_regularization_strength=0.1,l2_regularization_strength=0.1).minimize(s.loss)
				#s.train = tf.train.ProximalAdagradOptimizer(s.learning_rate,initial_accumulator_value=0.1,l1_regularization_strength=0.1,l2_regularization_strength=0.1).minimize(s.loss)
				s.train = tf.train.RMSPropOptimizer(s.learning_rate,decay=0.998,momentum=0.9,epsilon=1e-10).minimize(s.loss)
 				#decayed_rate = tf.train.exponential_decay(s.learning_rate, global_step, 600, 0.998, True)       
				#s.train = tf.train.MomentumOptimizer(decayed_rate, 0.9).minimize(s.loss)

			with tf.name_scope('prediction'):
				s.softmax = tf.nn.softmax(s.output, name="softmax")
				s.prediction = tf.cast(tf.argmax(s.softmax, 1), tf.int32)
				s.pred_correct = tf.equal(s.y_placeholder, s.prediction)
				s.pred_accuracy = tf.reduce_mean(tf.cast(s.pred_correct, tf.float32)) 

			s.init = tf.global_variables_initializer()
		s.session = tf.Session(graph=graph)
		s.session.run(s.init)
		s.writer = tf.summary.FileWriter('CIFAR10CNNTEST',graph=graph)

	def fit(s, train_dict):
		return s.session.run([s.loss, s.inc_step, s.pred_accuracy, s.train, s.merged], feed_dict=train_dict)

	def predict(s, test_dict):
		return s.session.run([s.pred_correct, s.prediction], feed_dict=test_dict)

In [0]:
import helpers_04
(train_data, train_labels,
            test_data, test_labels), readable_labels = helpers_04.load_cifar()



In [0]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
test_data = mnist.test.images  # Returns np.array
test_labels = np.asarray(mnist.test.labels, dtype=np.int32)

In [22]:
model = MNIST_Model()

print("{:3} {:>15s}".format("i", "Training MSE"))
for epoch in range(100):
	t0=time.clock()
	for batch_data, batch_labels in batches(train_data, train_labels, 100):
		m = min(.5 + epoch * 0.001, 0.99)
		train_dict = {  model.x_placeholder: batch_data, model.y_placeholder:batch_labels, 
						model.learning_rate: 0.001, model.momentum: m,
						model.x_keep_prob : 1, model.dropout_rate : 0.2 , model.dropout_boolean: True}

		loss, step, accuracy, _, summaries= model.fit(train_dict)
		model.writer.add_summary(summaries, global_step=step)
		model.writer.flush()
    
	t3=time.clock()-t0
	info_update = "Epoch: {:2d} Step: {:5d} Loss: {:8.2f} Acc: {:5.2f} time: {:5.2f}"
	print(info_update.format(epoch, step, loss, accuracy, t3))

i      Training MSE
Epoch:  0 Step:   500 Loss:     2.30 Acc:  0.14 time:  7.42
Epoch:  1 Step:  1000 Loss:     2.28 Acc:  0.14 time:  7.19
Epoch:  2 Step:  1500 Loss:     2.06 Acc:  0.28 time:  7.25
Epoch:  3 Step:  2000 Loss:     2.24 Acc:  0.13 time:  7.27
Epoch:  4 Step:  2500 Loss:     2.00 Acc:  0.20 time:  7.17
Epoch:  5 Step:  3000 Loss:     1.93 Acc:  0.28 time:  7.15
Epoch:  6 Step:  3500 Loss:     1.83 Acc:  0.28 time:  7.20
Epoch:  7 Step:  4000 Loss:     2.01 Acc:  0.24 time:  7.18
Epoch:  8 Step:  4500 Loss:     1.99 Acc:  0.27 time:  7.21
Epoch:  9 Step:  5000 Loss:     2.14 Acc:  0.18 time:  7.21
Epoch: 10 Step:  5500 Loss:     2.07 Acc:  0.25 time:  7.15
Epoch: 11 Step:  6000 Loss:     2.16 Acc:  0.18 time:  7.14
Epoch: 12 Step:  6500 Loss:     2.27 Acc:  0.10 time:  7.16
Epoch: 13 Step:  7000 Loss:     2.29 Acc:  0.16 time:  7.16
Epoch: 14 Step:  7500 Loss:     2.31 Acc:  0.12 time:  7.17
Epoch: 15 Step:  8000 Loss:     2.30 Acc:  0.11 time:  7.27
Epoch: 16 Step:  850

Epoch: 56 Step: 28500 Loss:     2.30 Acc:  0.12 time:  7.24
Epoch: 57 Step: 29000 Loss:     2.31 Acc:  0.09 time:  7.29
Epoch: 58 Step: 29500 Loss:     2.30 Acc:  0.11 time:  7.22
Epoch: 59 Step: 30000 Loss:     2.30 Acc:  0.10 time:  7.30
Epoch: 60 Step: 30500 Loss:     2.31 Acc:  0.12 time:  7.28
Epoch: 61 Step: 31000 Loss:     2.30 Acc:  0.07 time:  7.23
Epoch: 62 Step: 31500 Loss:     2.30 Acc:  0.17 time:  7.29
Epoch: 63 Step: 32000 Loss:     2.30 Acc:  0.10 time:  7.26
Epoch: 64 Step: 32500 Loss:     2.30 Acc:  0.14 time:  7.23
Epoch: 65 Step: 33000 Loss:     2.31 Acc:  0.06 time:  7.25
Epoch: 66 Step: 33500 Loss:     2.30 Acc:  0.13 time:  7.24
Epoch: 67 Step: 34000 Loss:     2.30 Acc:  0.14 time:  7.17
Epoch: 68 Step: 34500 Loss:     2.30 Acc:  0.08 time:  7.20
Epoch: 69 Step: 35000 Loss:     2.30 Acc:  0.09 time:  7.20
Epoch: 70 Step: 35500 Loss:     2.30 Acc:  0.09 time:  7.18
Epoch: 71 Step: 36000 Loss:     2.30 Acc:  0.11 time:  7.21
Epoch: 72 Step: 36500 Loss:     2.31 Acc

In [23]:
# Get test accuracy
batch_correct_cts = []
for batch_data, batch_labels in batches(test_data, test_labels, 200):
	test_dict = {model.x_placeholder : batch_data, model.y_placeholder : batch_labels, model.x_keep_prob : 1, model.dropout_rate : 1, model.dropout_boolean: False}
	correctness, curr_preds = model.predict(test_dict)
	batch_correct_cts.append(correctness.sum())

print(sum(batch_correct_cts) / len(test_data))

0.1
